In [4]:
pip install simpy

In [7]:
import simpy
import random
random.seed(42)
AVG_TIME_BETWEEN_ARRIVALS = 5*60
CONTAINERS_PER_VESSEL = 150
CRANE_MOVE_TIME = 3
TRUCK_TRIP_TIME = 6
NUM_BERTHS = 2
NUM_CRANES = 2
NUM_TRUCKS = 3
SIMULATION_TIME = 2000

In [8]:
class Terminal:
    def __init__(self, env):
        self.env = env
        self.berths = simpy.Resource(env, NUM_BERTHS)
        self.cranes = simpy.Resource(env, NUM_CRANES)
        self.trucks = simpy.Resource(env, NUM_TRUCKS)

In [9]:
def vessel(env, name, terminal):
    arrival_time = env.now
    print(f'{arrival_time:.2f}: {name} arrives at the terminal.')
    with terminal.berths.request() as req:
        yield req
        berth_time = env.now
        print(f'{berth_time:.2f}: {name} berths at the terminal.')
        for i in range(CONTAINERS_PER_VESSEL):
            with terminal.cranes.request() as crane_req:
                yield crane_req
                with terminal.trucks.request() as truck_req:
                    yield truck_req
                    yield env.timeout(CRANE_MOVE_TIME)
                    yield env.timeout(TRUCK_TRIP_TIME)

                    unload_time = env.now
                    print(f'{unload_time:.2f}: {name} unloads container {i+1}/{CONTAINERS_PER_VESSEL}.')

        leave_time = env.now
        print(f'{leave_time:.2f}: {name} Vessel leaves the terminal.')

In [10]:
def generate_vessels(env, terminal):
    vessel_id = 0
    while True:
        yield env.timeout(random.expovariate(1 / AVG_TIME_BETWEEN_ARRIVALS))
        vessel_id += 1
        env.process(vessel(env, f'Vessel {vessel_id}', terminal))

In [11]:
# we are starting the simulation
env = simpy.Environment()
terminal = Terminal(env)
env.process(generate_vessels(env, terminal))
env.run(until=SIMULATION_TIME)

306.02: Vessel 1 arrives at the terminal.
306.02: Vessel 1 berths at the terminal.
313.62: Vessel 2 arrives at the terminal.
313.62: Vessel 2 berths at the terminal.
315.02: Vessel 1 unloads container 1/150.
322.62: Vessel 2 unloads container 1/150.
324.02: Vessel 1 unloads container 2/150.
331.62: Vessel 2 unloads container 2/150.
333.02: Vessel 1 unloads container 3/150.
340.62: Vessel 2 unloads container 3/150.
342.02: Vessel 1 unloads container 4/150.
349.62: Vessel 2 unloads container 4/150.
351.02: Vessel 1 unloads container 5/150.
358.62: Vessel 2 unloads container 5/150.
360.02: Vessel 1 unloads container 6/150.
367.62: Vessel 2 unloads container 6/150.
369.02: Vessel 1 unloads container 7/150.
376.62: Vessel 2 unloads container 7/150.
378.02: Vessel 1 unloads container 8/150.
385.62: Vessel 2 unloads container 8/150.
387.02: Vessel 1 unloads container 9/150.
394.62: Vessel 2 unloads container 9/150.
396.02: Vessel 1 unloads container 10/150.
403.62: Vessel 2 unloads container 